# Object-Oriented Programming (III)

## Class Interface Techniques
#### Abstact class
일부가 미구현 된 method(**abstract method**)가 존재한다. Class instance를 만들어 사용하려는 목적이 아니라 상속할 틀을 만드는 super class를 말한다.  

Abstract class는 
- 구현은 subclass에게 위임한다. 
- 후손이 동일한 interface를 유지하기 위한 목적.

In [ ]:
class Super:
    """An abstract class
    to be implemented later"""
    def method(self):
        print('in Super.method') # Default behavior
    def delegate(self):
        self.action()            
    def action(self):            # Not implement at this moment
        raise NotImplementedError('action: not implemented')

s = Super()
s.method()
# s.delegate()   # Error

#### 상속자
조상으로 부터 모든 attribute과 method를 상속받아 사용한다. 
- 본인이 이룬게 없다. 모든 것이 조상 덕...

In [ ]:
class Inheritor(Super):
    pass

i = Inheritor()
i.method()  # call super method

#### 대체자
조상으로 부터 상속 받되 일부 method는 대체한다. 
- 조상의 method를 자손이 완전히 Override.

In [ ]:
class Replacer(Super):
    def method(self):
        print('in Replace.method')

r = Replacer()
s = Super()
for obj in (r, s):
    obj.method()

#### 확장자
조상으로 부터 상속받은 method를 활용하되 더 확장하여 발전시킨다.
- 조상 method를 이용하지만, 확장하여 override

In [ ]:
class Extender(Super):
    def method(self):
        print('Starting Extender.method')
        Super.method(self)
#         super().method()   # super() returns super class instance
        print('ending Extender.method')

e = Extender()
e.method()

#### 제공자
조상(abstact class)이 이루지 못한(구현을 위임한) method를 구현하여 제공한다.
- implementing `delegate` method by providing `action` method

In [ ]:
class Provider(Super):
    def action(self):
        print('in Provider.action')
        
p = Provider()
p.method()
p.delegate()

참고: **Polymorphism (다형성)**
> 하나의 interface를 통해 서로 다른 여러 타입을 제공하는 것을 의미한다.<br> 
> 보통 OOP 에서 말하는 다형성은 클래스에 선언된 method가 subclass에서 같은 이름으로 **overriding** 되어 여러 형태로 동작함을 의미한다.

주의:  **function overloading in Python**
> C나 Java에서는 function signature(function name, parameter type과 갯수)가 다르면
다른 function이다. 다시 말해, function name을 같이 사용하지만 사실은 언어적으로
다른 function (signature)인 것이다. 

Python에서 다음과 같이 적으면 meth는 마지막 define한 meth만 유효 뿐이다.

```Python
class C:
    def meth(self, x):
        ...
    def meth(self, x, y, z):  # 이것이 유효하다.
        ...

c = C()
c.meth(1)        #  TypeError. 파라미터 y, z가 없다.   
c.meth(1, 2, 3)  # OK
```

1) Default argument value로 쓰거나 
```Python
    def meth(self, x, y=0, z=None)
```
2) 다음과 같이 Variable argument list로 받아야 한다.

In [ ]:
#Use variable arguments list, instead.
class C:
    def meth(self, *args):
        print(args)
        for arg in args:
            if isinstance(arg, int):
                print(f'{arg}: handle int')
            elif isinstance(arg, str):
                print(f'{arg}: handle str')
                           
c = C()
c.meth(1, 'a', 'b')

### A stream processor

In [ ]:
from abc import ABCMeta, abstractmethod

# Abstract class
class Processor(metaclass=ABCMeta):
    def __init__(self, reader, writer):
        self.reader = reader
        self.writer = writer
    
    def process(self):
        for line in self.reader:
            data = self.converter(line)
            self.writer.write(data)
    
    @abstractmethod        
    def converter(self, data):       # delegates implementation
        raise NotImplemented('converter must be defined')

In [ ]:
# Provider overriding converter()
class UpperCase(Processor):
    def converter(self, data):
        return data.upper()
    
class ToKorean(Processor):
    korean_dict = {'a': '하나의', 'abacus': '곁눈질', 
                   'abandon': '버리다', 'abandonment': '포기' }
    def converter(self, data):
        return ToKorean.korean_dict.get(data.strip(), "<unknown>") + '\n'
    
import sys, io

buf = '\n'.join(['a', 'abacus', 'abandon', 'abandoned']) + '\n'
file = io.StringIO(buf)   # file-like object in memory buffer
upper = UpperCase(file, sys.stdout)
upper.process()

file.seek(0)        # rewind the file
capital = ToKorean(file, sys.stdout)
capital.process()

### Example: Extending Types by Embedding
list object을 이용하여 `Set` class 만들기

In [ ]:
class Set:
    def __init__(self, value = []):    # Constructor
        self.data = []                 # Manages a list
        self.concat(value)

    def intersection(self, other):        # other is any sequence
        res = []                       # self is the subject
        for x in self.data:
            if x in other:             # Pick common items
                res.append(x)
        return Set(res)                # Return a new Set

    def union(self, other):            # other is any sequence
        res = self.data[:]             # Copy of my list
        for x in other:                # Add items in other
            if not x in res:
                res.append(x)
        return Set(res)

    def concat(self, value):
        for x in value:                
            if not x in self.data:     # Removes duplicates
                self.data.append(x)

    def __len__(self):          return len(self.data)        # len(self)
    def __getitem__(self, key): return self.data[key]        # self[i], self[i:j]
    def __and__(self, other):   return self.intersection(other) # self & other
    def __or__(self, other):    return self.union(other)     # self | other
    def __repr__(self):         return 'Set({})'.format(repr(self.data))  
    def __iter__(self):         return iter(self.data)       # for x in self:

    
x = Set([1,3,5,7, 1, 3])
y = Set([2,1,4,5,6])
print(x, y, len(x))
print(x.intersection(y), y.union(x))
print(x & y, x | y)
print(x[2], y[:2])
for element in x:
    print(element, end=' ')
print()
print(3 not in y)  # membership test
print(list(x))   # convert to list because x is iterable

## Exercise. Set class에 methods 추가
Python `set` type에 쓰이는 다음의 method나 operator를 Set class에 추가하고, `set` 처럼 쓰이는지 시험하라.

self.issubset(other)

self <= other
> Test whether every element in the set is in other.

self < other
> Test whether the set is a proper subset of other, that is, set <= other and set != other.

self.issuperset(other)

self >= other
> Test whether every element in other is in the set.

self > other

>Test whether the set is a proper superset of other, that is, set >= other and set != other.

self |= other
> Update the set, adding elements from all others.

self.intersection_update(others)

self &= other
> Update the set, keeping only elements found in it and all others.

self.difference_update(others)

self -= other
> Update the set, removing elements found in others.

self.symmetric_difference_update(other)

self ^= other
> Update the set, keeping only elements found in either set, but not in both.

self.add(elem)

> Add element elem to the set.

self.remove(elem)

> Remove element elem from the set. Raises KeyError if elem is not contained in the set.